## extracting from recon3d

In [23]:
import os
import re
import warnings
from cobra.io.json import load_json_model as load_json_cobra_model
import escher
import mass
import numpy as np
import pandas as pd
import equilibrator_api
from equilibrator_api import ComponentContribution, Q_
import sympy as sym
from cobra import Model, Reaction, Metabolite
import cobra.test
from os.path import join
from mass.util import qcqa
from mass.util import qcqa_model
from cobra import DictList
from mass import (
    MassConfiguration, MassMetabolite, MassModel,
    MassReaction, Simulation, UnitDefinition)
from mass.io.json import save_json_model as save_json_mass_model
from mass.visualization import plot_comparison, plot_time_profile
from mass.visualization import (
    plot_ensemble_phase_portrait, plot_ensemble_time_profile)
mass_config = MassConfiguration()
mass_config.irreversible_Keq = float("inf")
print("MASSpy version: {0}".format(mass.__version__))
from six import iteritems
import matplotlib.pyplot as plt
from mass.thermo import (
    ConcSolver, sample_concentrations,
    update_model_with_concentration_solution)
from cobra.io.json import *
import cobra_dict as c_d
import csv
from cobra_dict import metabolite_to_dict 
from cobra_dict import reaction_to_dict 


MASSpy version: 0.1.6


In [24]:
#load Recon3D
model_dir = os.path.abspath("models")
data_dir = os.path.abspath("data")
model_dir
R3D=load_json_cobra_model(filename=os.path.join(model_dir,"Recon3D.json"))

In [25]:
R3D.reactions.ADK1

Reaction identifier,ADK1
Name,Adenylate kinase
Memory address,0x0250348e5550
Stoichiometry,amp_c + atp_c <=> 2.0 adp_c AMP C10H12N5O7P + ATP C10H12N5O13P3 <=> 2.0 ADP C10H12N5O10P2
GPR,122481_AT1 or 203_AT1 or 26289_AT1 or 26289_AT2 or 204_AT1
Lower bound,-1000.0
Upper bound,1000.0


In [26]:
reaction_list = []
reaction_list = ['HEX1',
                'PGI',
                'FBP',
                'PFK',
                'FBA',
                'TPI',
                'GAPD',
                'PGK',
                'PGM',
                'ENO',
                # 'PEPtm',
                'PEPCKm',
                'PYK',
                'PCm',
                'LDH_L',
                
                'PGMT',
                "GLPASE2",
                'GLDBRAN',
                "GLPASE1",
                'GLGNS1',
                'GGNG',
                'GALUi',
                # 'CO2tm',
                # 'Htm',
                # 'O2tm',
                #  'H2Otm',
                 
#                 'G6Pter',
#                 'G6PPer',
#                 'GLCter',
#                 'GLCt1',
                # 'PYRt2m', 
#                 'H2Oter', 
#                 'PIter', 
#                 'H2Ot', 
#                 'PIt', 
#                 'Ht', 
#                 'L_LACt2r', #lactose transport between lactate in cytosol and extracellular 
#                  'PYRt2',
                'ADK1',
                'ATPM',
#                 'DM_nadh'
                ]

In [27]:
rxn_df=pd.DataFrame()
for rxn in reaction_list:
    r3d_rxn_temp= R3D.reactions.get_by_id(rxn)
    r3d_rxn = r3d_rxn_temp.copy()
    for m in r3d_rxn.metabolites:
        # print(m)
        m.id= m.id.rpartition("_" + str(m.compartment))[0]
#         # m.compartment = None
    r=reaction_to_dict(r3d_rxn)
# #     print(r)
    df=pd.DataFrame.from_dict(r,orient='index')
    df=df.T
    rxn_df = pd.concat([rxn_df,df ])

rxn_df=rxn_df.set_index('id')
rxn_df

,name,metabolites,lower_bound,upper_bound,gene_reaction_rule,subsystem,notes,annotation
id,,,,,,,,
HEX1,Hexokinase (D-glucose:ATP),"{'adp': 1.0, 'atp': -1.0, 'g6p': 1.0, 'glc__D'...",0.0,1000.0,3098_AT3 or 80201_AT1 or 2645_AT3 or 3098_AT4 ...,Glycolysis/gluconeogenesis,{'original_bigg_ids': ['HEX1']},"{'bigg.reaction': ['HEX1'], 'biocyc': ['META:G..."
PGI,Glucose-6-phosphate isomerase,"{'f6p': 1.0, 'g6p': -1.0}",-1000.0,1000.0,2821_AT1,Glycolysis/gluconeogenesis,{'original_bigg_ids': ['PGI']},"{'bigg.reaction': ['PGI'], 'biocyc': ['META:PG..."
FBP,Fructose-bisphosphatase,"{'f6p': 1.0, 'fdp': -1.0, 'h2o': -1.0, 'pi': 1.0}",0.0,1000.0,2203_AT1 or 8789_AT1,Glycolysis/gluconeogenesis,{'original_bigg_ids': ['FBP']},"{'bigg.reaction': ['FBP'], 'ec-code': ['3.1.3...."
PFK,Phosphofructokinase,"{'adp': 1.0, 'atp': -1.0, 'f6p': -1.0, 'fdp': ...",0.0,1000.0,5214_AT1 or 5211_AT2 or (5214_AT1 and 5211_AT1...,Glycolysis/gluconeogenesis,{'original_bigg_ids': ['PFK']},"{'bigg.reaction': ['PFK'], 'ec-code': ['2.7.1...."
FBA,Fructose-bisphosphate aldolase,"{'dhap': 1.0, 'fdp': -1.0, 'g3p': 1.0}",-1000.0,1000.0,226_AT2 or 226_AT1 or 226_AT3 or 230_AT1 or 22...,Glycolysis/gluconeogenesis,{'original_bigg_ids': ['FBA']},"{'bigg.reaction': ['FBA'], 'ec-code': ['4.1.2...."
TPI,Triose-phosphate isomerase,"{'dhap': -1.0, 'g3p': 1.0}",-1000.0,1000.0,7167_AT1 or 286016_AT1,Glycolysis/gluconeogenesis,{'original_bigg_ids': ['TPI']},"{'bigg.reaction': ['TPI'], 'biocyc': ['META:TR..."
GAPD,Glyceraldehyde-3-phosphate dehydrogenase,"{'13dpg': 1.0, 'g3p': -1.0, 'h': 1.0, 'nad': -...",-1000.0,1000.0,2597_AT1 or 26330_AT1,Glycolysis/gluconeogenesis,{'original_bigg_ids': ['GAPD']},"{'bigg.reaction': ['GAPD'], 'biocyc': ['META:G..."
PGK,Phosphoglycerate kinase,"{'13dpg': 1.0, '3pg': -1.0, 'adp': 1.0, 'atp':...",-1000.0,1000.0,5230_AT1 or 375056_AT1 or 5232_AT1,Glycolysis/gluconeogenesis,{'original_bigg_ids': ['PGK']},"{'bigg.reaction': ['PGK'], 'biocyc': ['META:PH..."
PGM,Phosphoglycerate mutase,"{'2pg': -1.0, '3pg': 1.0}",-1000.0,1000.0,669_AT1 or 5223_AT1 or 5224_AT2 or 5224_AT1 or...,Glycolysis/gluconeogenesis,{'original_bigg_ids': ['PGM']},"{'bigg.reaction': ['PGM'], 'biocyc': ['META:3P..."


In [28]:
metabolite_list=[]
met_df=pd.DataFrame()

for reaction in reaction_list:
    rxn_temp = R3D.reactions.get_by_id(reaction)
    rxn = rxn_temp.copy()
    for met in rxn.metabolites:
        r3d_met= R3D.metabolites.get_by_id(met.id)
        met_temp = r3d_met.copy()
        new_id = met_temp.id.rpartition("_" + str(met_temp.compartment))[0]
        if new_id not in metabolite_list:
            met_temp.id = met_temp.id.rpartition("_" + str(met_temp.compartment))[0]
            metabolite_list.append(met_temp.id)
            met_temp.compartment = None
            m=metabolite_to_dict(met_temp)
            df_2=pd.DataFrame.from_dict(m,orient='index')
            df_2=df_2.T
            # met_df=met_df.append(df_2)
            met_df = pd.concat([df_2, met_df])

    # print(rxn.id)

met_df=met_df.set_index('id').drop("compartment", axis=1)
met_df

,name,charge,formula
id,,,
amp,AMP C10H12N5O7P,-2,C10H12N5O7P
utp,UTP C9H11N2O15P3,-4,C9H11N2O15P3
ppi,Diphosphate,-3,HO7P2
udpg,UDPglucose,-2,C15H22N2O17P2
udp,UDP C9H11N2O12P2,-3,C9H11N2O12P2
glygn1,"Glycogen, structure 1 (glycogenin-11[1,4-Glc])",0,C66H111O56X
ggn,"Primed glycogenin (glycogenin-8[1,4-Glc])",0,C48H81O41X
glygn2,"Glycogen, structure 2 (glycogenin-1,6-{7[1,4-G...",0,C66H111O56X
dxtrn,"Phosphorylase-limit dextrin (glycogenin-1,6{4[...",0,C48H81O41X


In [29]:
csv_met = os.path.join(data_dir,"gly_gluco_glycogen_no_compts_met_df.csv")
met_df.to_csv(csv_met)

In [30]:
# csv_rxn = os.path.join(data_dir,"gly_gluco_reaction_df.csv")
# rxn_df.to_csv(csv_rxn)

In [31]:
json_rxn = os.path.join(data_dir,"gly_gluco_glycogen_no_compt_reaction_df.json")
rxn_df.to_json(json_rxn
,orient="index"
)
    # orient="columns"
    